# Домашнее задание 2 (5 баллов).

*Все задания ниже имеют равный вес (5/10)*

Код для импорта мы написали за вас (не благодарите, нам не трудно). Дальше код будете писать вы. 

[Тут](https://habr.com/ru/companies/ruvds/articles/494720/) шпора по pandas. За основу домашнего задания взят ноутбук [отсюда](https://rutube.ru/video/f884aa6ed5f94120b7304506042fe5bb/) (не подглядывайте!).

In [1]:
import pprint

import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 9)
pd.set_option('display.width', 250)

#### Описание данных

Автор д/з - плохой человек, который не стал переводить описание с мотивировкой, что весь DS на английском. Так что описание полей будет на английском:

1. Account ID
- Description: A unique identifier for each social media account in the dataset.
- Type: Integer
- Example: 1, 2, 3, …
2. Username
- Description: The username or handle of the social media account.
- Type: String
- Example: john_doe, tech_guru_22, fitness_freak
3. Platform
- Description: The social media platform the account is using (Instagram, Twitter, Facebook, TikTok, LinkedIn).
- Type: Categorical (String)
- Example: Instagram, Twitter, Facebook, TikTok, LinkedIn
4. Follower Count
- Description: The total number of followers the account has.
- Type: Integer
- Example: 1500, 245000, 78000
5. Posts Per Week
- Description: The average number of posts the account creates per week.
- Type: Integer
- Example: 3, 5, 7
6. Engagement Rate
- Description: The percentage of interactions (likes, comments, shares) relative to the follower count. This is a measure of how engaging the content is.
- Type: Float
- Range: 0.01 to 0.15
- Example: 0.045 (4.5% engagement rate)
7. Ad Spend (USD)
- Description: The monthly amount spent on advertising or promoting posts.
- Type: Float
- Example: 150.75, 850.00, 300.50
8. Conversion Rate
- Description: The percentage of users who take a desired action (e.g., clicking a link, signing up, etc.) after interacting with an ad.
- Type: Float
- Range: 0.01 to 0.05 (1% to 5% conversion rate)
- Example: 0.025 (2.5% conversion rate)
9. Campaign Reach
- Description: The total number of unique users reached by the user’s campaigns in a given month.
- Type: Integer
- Example: 5000, 20000, 15000

#### Задание 0

Подгрузите данные. Да-да, за чтение таблицы баллов не будет))

**Hint**: [pd.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)

In [2]:
df = pd.read_csv("data.csv")

#### Задание 1

Колонка `Platform` содержит название различных платформ. Давайте представим, что в них есть некоторое отношение порядка. Закодируйте каждую платформу целым числом (от 0 до N) и положите этот "код" в новую колонку `Platform_Code`. Теперь вычислите корреляцию Спирмена между всеми парами колонок в датасете (результатом будет таблица корреляций). В качестве ответа выведите значение корреляции `Platform_Code` с `Engagement Rate`. Можете после вывода числа еще коротко написать, что оно означает (нет, это не оценивается).

**Hint**: [pd.factorize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html), [pd.DataFrame.select_dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.select_dtypes.html), [pd.DataFrame.corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html).

In [3]:
codes, types = pd.factorize(df["Platform"])
df["Platform_Code"] = codes
corr_spearman = df.select_dtypes(include=['number']).corr(method="spearman")
corr_spearman['Platform_Code']['Engagement Rate']

0.03138169529349812

Слабая корреляция, очень, практически отсутствует

#### Задание 2

Теперь посмотрите на столбец `Follower Count`. В нем какие-то числа. Иногда бывает полезно провести дискретизацию такого признака. Разбейте все значения в столбце на 4 группы: "Low", "Medium", "High", "Very High". Каждая группа включает в себя новые 25% данных. То есть, Low включает в себя 25% самых маленьких значений признака и так далее. Положите значения "Low", "Medium", "High" или "Very High" для каждого сэмпла датасета в новую колонку `Follower_Bin`. Теперь посчитайте среднее значение `Engagement Rate` для каждой категории из `Follower_Bin`. В качестве ответа выведите значение для категории "High".

**Hint**: [pd.qcut](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html), [pd.groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html), [pd.DataFrame.mean](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mean.html)

In [4]:
df["Follower_Bin"] = pd.qcut(df["Follower Count"], 4, labels=["Low", "Medium", "High", "Very High"])
print(df.groupby("Follower_Bin")["Engagement Rate"].median()["High"])

0.0856


C:\Users\Dmitriy\AppData\Local\Temp\ipykernel_13108\1303953712.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby("Follower_Bin")["Engagement Rate"].median()["High"])


#### Задание 3

Иногда бывает полезно превратить широкую таблицу в длинную (например, для визуализаций сразу нескольких признаков на одной картинке). Да, звучит странно, но именно этим вы сейчас и займетесь. Сделайте новый датафрейм `melted_df`, в который вы поместите каждый сэмпл датасета 6 раз: по одному разу на значение из 'Follower Count', 'Posts Per Week', 'Ad Spend (USD)', 'Conversion Rate', 'Engagement Rate' и 'Campaign Reach'. То есть, вы берете сэмпл из датасета (строку) и превращаете ее в 6 отдельных строк. Каждая отдельная строка в столбце `Metric` имеет имя из предложенного списка 5 признаков, а в столбце `Value` - значение данного сэмпла по этому признаку. Значение `Platform` повторяется в этих 6 строках.

Иначе говоря, 

```json
{
    "Account ID": 1,
    "Username": "harrislisa",
    "Platform": "TikTok",
    "Follower Count": 54217,
    "Posts Per Week": 3,
    "Engagement Rate": 0.0986,
    "Ad Spend (USD)": 538.1,
    "Conversion Rate": 0.049,
    "Campaign Reach": 1308,
    "Platform_Code": 0,
    "Follower_Bin": "Low"
}
```

превращается в 

```json
{
    "Platform": "TikTok",
    "Metric": "Follower Count",
    "Value": 54217,
},
{
    "Platform": "TikTok",
    "Metric": "Posts Per Week",
    "Value": 3,
}, ...
```

Для каждого уникальной пары значений (`Platform`, `Metric`) посчитайте моду среди всех значений `Value` для этой пары, результат сделайте списком и оставьте только наибольшее. В качестве ответа выведите сумму полученных мод (сумму всех значений в столбце `Value` уже после вычисления мод). Иначе говоря, выведите сумму всех мод значений для всех уникальных пар (`Platform`, `Metric`).

**Hint**: [pd.melt](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.melt.html), [pd.DataFrame.mode](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.mode.html), [pd.DataFrameGroupBy.agg](https://pandas.pydata.org/docs/dev/reference/api/pandas.core.groupby.DataFrameGroupBy.agg.html)

In [5]:
melted_df = pd.melt(df, id_vars=['Platform'], var_name="Metric", value_name="Value", value_vars=['Follower Count', 'Posts Per Week', 'Ad Spend (USD)', 'Conversion Rate', 'Engagement Rate','Campaign Reach'])
grouped_melted_df = melted_df.groupby(["Platform", "Metric"])
modes = grouped_melted_df.apply(lambda x: x.mode().max())
print(modes.sum())
# поч так сложно для понимания... Формулировку придумывал сам сатана, отвечаю

Value    3.100285e+06
dtype: float64


#### Задание 4

А теперь хочется посмотреть на самые популярные аккаунты на разных платформах. Для каждой платформы отсортируйте датафрейм по убыванию количества подписчиков (`Follower Count`) - да, без циклов, сразу для всех платформ сделать сортировку, а затем оставьте только первые три записи для каждой платформы - это и будут три самых популярных аккаунта для каждой платформы. В качестве ответа выведите саму таблицу и минимальное значение `Follower Count` в ней.

**Hint**: к *groupby* можно применять функции - это эквивалентно применению функции к каждой "группе" внутри groupby-объекта. Читайте [про применение apply к датафрейму после groupby](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html#flexible-apply).

In [6]:
table_top_3 = df.sort_values("Follower Count", ascending=False).groupby("Platform").head(3)
# table_top_3 = df.groupby("Platform", group_keys=False).apply(lambda x: x.nlargest(3, "Follower Count"))
print(table_top_3, table_top_3["Follower Count"].min())

      Account ID         Username   Platform  Follower Count  ...  Conversion Rate  Campaign Reach  Platform_Code  Follower_Bin
2403        2404           eric65   Facebook          999982  ...           0.0281           17312              2     Very High
4920        4921      teresaellis    Twitter          999919  ...           0.0460            3975              4     Very High
5838        5839     edwardthomas     TikTok          999739  ...           0.0325           35523              0     Very High
8685        8686  alexandersamuel  Instagram          999726  ...           0.0205           11050              3     Very High
9684        9685           sriley    Twitter          999442  ...           0.0225           12783              4     Very High
3965        3966         lrodgers  Instagram          999351  ...           0.0335           12391              3     Very High
4234        4235    andradewesley     TikTok          999234  ...           0.0481           17188      

#### Задание 5

Хочется посчитать какую-то метрику. Мы хотим посмотреть, на отношение разности суммы подписчиков аккаунтов с высокой и низкой конверсией к суммарному охвату рекламы на каждой платформе. То есть, мы делим аккаунты на две группы: высокая и низка конверсия. Затем мы смотрим на то, на сколько сильно влияние аккаунтов с высокой конверсией по сравнению с аккаунтами с низкой конверсией. 

Давайте определим *Conversion Influence* следущим образом:

$$Conversion Influence = \frac{Total Follower\ Count (High) - Total Follower\ Count (Low)}{Total Campaign Reach (High)+Total Campaign Reach (Low)}$$

Считать эту метрику мы будет для каждой `Platform`. В этой формуле High - это значения всех сэмплов датасета, в которых `Conversion Rate` больше медианы, а `Low` - не более медианы. `Total Feature` - это суммарное количество значений `Feature` либо по `High` сэмплам, либо по `Low`.

Чтобы постоянно не пересчитывать, где High. где Low, сделайте новую колонку в датасете `Conversion_Category`. Положите в нее для каждой строки либо High, либо Low.

Выведите платформу с самым большим `Conversion Influence`.

**Hint**: данное задание не про *groupby*, а скорее про [pd.pivot_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html). Сделайте сводную таблицу, по которой уже можно посчитать суммы, а затем подставить их в формулы.

In [7]:
med_conversion_rate = df["Conversion Rate"].median()
df["Conversion_conversion_rate"] = df["Conversion Rate"].apply(lambda x: "High" if x >= med_conversion_rate else "Low")
df_platforms = pd.pivot_table(df, values=["Follower Count", "Campaign Reach"], index="Platform", columns=["Conversion_conversion_rate"], aggfunc="sum")
df_platforms["ConversionInfluence"] = ((df_platforms["Follower Count", "High"] - df_platforms["Follower Count", "Low"]) / 
                                       (df_platforms["Campaign Reach", "High"] + df_platforms["Campaign Reach", "Low"]))
print(df_platforms["ConversionInfluence"].idxmax())

Twitter


#### Задание 6

Мы знаем, что вам понравилось считать метрики по формуле. Давайте закрепим этот успех. Теперь для каждой платформы посчитаем, на сколько эффективна реклама в разрезе трех последовательных записей в датасете. 

Для каждой платформы отсортируйте записи в порядке убывания `Posts Per Week`. Будто бы аккаунты, которые постят чаще, используют более "активные" стратегии по рекламе. Теперь посчитайте *скользущие суммы с окном 3* по `Campaign Reach` и `Ad Spend (USD)`. Скользящая сумма с окном N - это вы идете по массиву, берете все последовательные тройки записей и суммируете их. Для первых двух записей троек не найдется. Для них скользящее среднее - NaN, что нам не помешает. 

Теперь для каждого окна посчитайте 

$$Rolling Efficiency Ratio = \frac{Rolling Sum of Campaign Reach}{Rolling Sum of Ad Spend}$$

По сути, для каждого окна вы посчитаете сколько пользователе привлеклось за один доллар, потреченный на рекламу, в данном окне. Понятно, что значений будет столько, сколько окон. Нам интересно максимально значение такой эффективности для каждой платформы.

В качестве ответа выведите название платформы с наибольшей максимальной эффективность и наименьшей (два названия, не одно, не три, ровно два).

**Hint**: окна можно делать через [pd.DataFrame.rolling](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rolling.html).

In [8]:
df_sorted = df.sort_values(by=["Platform", "Posts Per Week"], ascending=[True, False])
# Вычисление скользящих сумм по двум фичам в !отдельности! - "Campaign Reach" и "Ad Spend (USD)"
# Сначала датасет сгруппируется по платформам, потом в каждой группе посчитаются роллинг суммы, потом мы разгруппируем, а правильные р.суммы для каждой строки останутся
df_sorted_with_rolling_sums = df_sorted.groupby("Platform")[["Campaign Reach", "Ad Spend (USD)"]].rolling(window=3).sum().reset_index()
df_sorted_with_rolling_sums["Rolling Efficiency Ratio"] = df_sorted_with_rolling_sums["Campaign Reach"] / df_sorted_with_rolling_sums["Ad Spend (USD)"]
print(df_sorted_with_rolling_sums)
# Мы должны вывести платформы с наименьшей и наибольшей "максимальной" эффективностью (т.е. лучшую из лучших и худшую из !лучших!, а не худшую из худших, как я думал предыдущие 2 часа решения...), поэтому сначала ищем все максимальные эффективности для каждой платформы
max_efficiencies = df_sorted_with_rolling_sums.groupby("Platform")["Rolling Efficiency Ratio"].max()
print(max_efficiencies, "\n")
print("Ответ:", max_efficiencies.idxmax(), max_efficiencies.idxmin())

      Platform  level_1  Campaign Reach  Ad Spend (USD)  Rolling Efficiency Ratio
0     Facebook       25             NaN             NaN                       NaN
1     Facebook       57             NaN             NaN                       NaN
2     Facebook       71         74760.0         1259.71                 59.346993
3     Facebook       76         67506.0         1631.14                 41.385779
4     Facebook      115         77597.0         1300.98                 59.645037
...        ...      ...             ...             ...                       ...
9995   Twitter     9909         98328.0         1938.06                 50.735271
9996   Twitter     9922         72763.0         1669.41                 43.586057
9997   Twitter     9926         67615.0         2292.51                 29.493874
9998   Twitter     9944         63033.0         2132.56                 29.557433
9999   Twitter     9962         50527.0         1744.73                 28.959782

[10000 rows x 5

#### Задание 7

Это еще не все прекрасные функции pandas, которые мы хотим вам показать. Теперь вы посчитаете, сколько аккаунтов на каждой платформе одновременно лучшие по `Engagement Rate` и `Conversion Rate`.

Сделайте два отдельных суб-сета. В одном оставьте для каждой платфмормы один топовый аккаунт по `Engagement Rate`, в другом - по `Conversion Rate`. Соедините эти два подмножества по столбцу `Platform` так, что в одной строке есть описание сразу двух аккаунтов-лидеров. Теперь посмотрите равны ли имена аккаунтов в одной строке. Выведите количество строк, в которых названия аккаунтов совпадают.

In [9]:
df_top_engagement = df.loc[df.groupby("Platform")["Engagement Rate"].idxmax()]
df_top_conversion = df.loc[df.groupby("Platform")["Conversion Rate"].idxmax()]
df_top = df_top_engagement.merge(df_top_conversion, on="Platform", how="left")
# Просто для читаемости очищаю ненужные колонки
df_top.drop(columns=["Follower Count_x", "Follower Count_y", "Campaign Reach_x", "Campaign Reach_y", "Follower_Bin_x", "Follower_Bin_y", "Conversion_conversion_rate_x", "Conversion_conversion_rate_y", "Posts Per Week_x", "Posts Per Week_y", "Ad Spend (USD)_x", "Ad Spend (USD)_y", "Platform_Code_x", "Platform_Code_y"], inplace=True)
print((df_top["Username_x"] == df_top["Username_y"]).sum())

0


#### Задание 8

Давайте теперь что-то попроще сделаем. Например, посчитаем отношение суммарного количества подписчиков на аккаунтах с высокой конверсией к такой же сумме в аккаунтах с низкой конверсией (очевидно, для каждой платформы). По сути, мы просто хотим получить число, которое характеризует, на сколько сильно аккаунты с высокой конверсией "доминируют" над аккаунтами с низкой конверсией в плане количества подписчиков.

Высокой конверсией будем считать конверсию больше средней. Остальное - низкая. Посчитайте суммы подписчиков для каждой платформы, поделите одно на другое и выведите разницу между самым большим значением и самым маленьким, а также платформы, которые соотвутствуют этим значениям.

Используйте магическую команду `%%time`, чтобы замерить, сколько времени ушло на исполнение вашего pandas-скрипта.

In [10]:
%%time

mid_conv = df["Conversion Rate"].mean()
df["Conv_class"] = df["Conversion Rate"].apply(lambda x: "High" if x >= mid_conv else "Low")
df_platforms = pd.pivot_table(df, values=["Follower Count"], index="Platform", columns=["Conv_class"], aggfunc="sum")
df_platforms["Conversion_ratio"] = df_platforms["Follower Count", "High"] / df_platforms["Follower Count", "Low"]
# print(df_platforms)

print("Ratio between max and min: ", df_platforms["Conversion_ratio"].max() - df_platforms["Conversion_ratio"].min())
print(f"This is conversion_ratio of [{df_platforms["Conversion_ratio"].idxmax()}] - conversion_ratio of [{df_platforms["Conversion_ratio"].idxmin()}]")

           Follower Count            Conversion_ratio
Conv_class           High        Low                 
Platform                                             
Facebook        491799506  520352686         0.945127
Instagram       468232352  544465076         0.859986
LinkedIn        490706904  518510323         0.946378
TikTok          482930355  504080020         0.958043
Twitter         518386631  499951697         1.036873
Ratio between max and min:  0.17688741338715763
This is conversion_ratio of [Twitter] - conversion_ratio of [Instagram]
CPU times: total: 0 ns
Wall time: 7.5 ms


#### Задание 9

А теперь решите задание 8 чисто питоном. Никаких функций и методов pandas. Только питоновские циклы. Замерьте время выполнения кода. Наконец, сравните время в задании 8 и 9. Напишите ниже, кто же победил: чистый python и pandas?

**Hint**: Чтобы итерироваться по датафрейму, можно из него сделать генератор через [pd.DataFrame.iterrows](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html) или [pd.DataFrame.itertuples](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.itertuples.html#pandas.DataFrame.itertuples). К слову, это не все способы итерироваться по датафрейму.

In [11]:
%%time

import pprint

def key_from_value(value, d: dict):
    for i in d.keys():
        if abs(d[i]["conv_ratio"] - value) < 0.00001: # потому что у нас вещественные переменные
            return i

data = df.drop(columns=["Campaign Reach", "Follower_Bin", "Conversion_conversion_rate", "Posts Per Week", "Ad Spend (USD)", "Platform_Code", "Username", "Account ID", "Engagement Rate", "Conv_class"]).to_dict(orient="records")
mid_conv = sum(row["Conversion Rate"] for row in data) / len(data)
platforms = set([i["Platform"] for i in data])
platforms_sums = {i: {"sum_high": 0, "sum_low": 0, "conv_ratio": None} for i in platforms}
for row in data:
    platforms_sums[row["Platform"]]["sum_high" if row["Conversion Rate"] >= mid_conv else "sum_low"] += row["Follower Count"]
for platform in platforms_sums.keys():
    platforms_sums[platform]["conv_ratio"] = platforms_sums[platform]["sum_high"] / platforms_sums[platform]["sum_low"]
# pprint.pprint(platforms_sums)
ratios = [platforms_sums[i]["conv_ratio"] for i in platforms_sums.keys()]
print("Ratio between max and min: ", max(ratios) - min(ratios))
print(f"This is conversion_ratio of [{key_from_value(max(ratios), platforms_sums)}] - conversion_ratio of [{key_from_value(min(ratios), platforms_sums)}]")

{'Facebook': {'conv_ratio': 0.9451272554784121,
              'sum_high': 491799506,
              'sum_low': 520352686},
 'Instagram': {'conv_ratio': 0.8599860168074398,
               'sum_high': 468232352,
               'sum_low': 544465076},
 'LinkedIn': {'conv_ratio': 0.9463782729741332,
              'sum_high': 490706904,
              'sum_low': 518510323},
 'TikTok': {'conv_ratio': 0.958043040468059,
            'sum_high': 482930355,
            'sum_low': 504080020},
 'Twitter': {'conv_ratio': 1.0368734301945974,
             'sum_high': 518386631,
             'sum_low': 499951697}}
Ratio between max and min:  0.17688741338715763
This is conversion_ratio of [Twitter] - conversion_ratio of [Instagram]
CPU times: total: 0 ns
Wall time: 10.5 ms


**А победителем является**: PANDAS

#### Задание 10

Крайне серьезное задание. Отнеситесь к нему соответствующе. В ячейке ниже напишите ваш любимый анекдот или мем (только без баянов, окей?). Можно плохие. Помните, это задание на полный балл. Проверяющий работу ассистент должен улыбнуться.

Если вставляете картинку, то убедитесь, что вы ее не подгружаете локально. А то будет неудобно - потерять балл на этом задании, когда надо было выложить картинку на облако и прокинуть ссылку. И нет, нельзя сюда просто ссылку вставить. Либо ищите, как вставить картинку, либо смешной анекдот. Есть всего два стула - выбирайте...

Я, конечно, не собака, но моя любимая команда - лежать... 
А вообще, pandas с sns, matplotlib'ом и numpy'ем за год смогли выжечь все мои эмоции, включая чувство юмора((( Извините...